In [ ]:
import numpy as np

In [ ]:
num_rotation = 7
num_level = 13
num_actual_level = 13
height = 1.0 / num_actual_level

In [ ]:
with open('../../../normalized_objs/models/dna_thick_1.obj', 'w') as f:
    for i in range(num_actual_level):
        z = i * height
        r = 3.5 / num_actual_level
        theta = (i % num_rotation) * 2 * np.pi / num_rotation
        x = r * np.cos(theta)
        y = r * np.sin(theta)

        mz = z + height / 3
        mx = -x
        my = -y
        f.write('v {} {} {}\n'.format(x, y, z))
        f.write('v {} {} {}\n'.format(mx, my, mz))
    for i in range(num_actual_level-1):
        f.write('l {} {}\n'.format(2*i + 1, 2 * i + 3))
        f.write('l {} {}\n'.format(2*i + 2, 2 * i + 4))
        
    for i in range(num_actual_level - 1)[1:]:
        f.write('l {} {}\n'.format(2*i + 1, 2 * i + 2))
        f.write('l {} {}\n'.format(2*i + 2, 2 * i + 3))


In [ ]:
normals = []
for i in range(num_level):
    theta = (i % num_rotation) * 2 * np.pi / num_rotation
    z = 0
    x = np.sin(theta)
    y = -np.cos(theta)
    normals.append([x, y, z])
    normals.append([x, y, z])

In [ ]:
normals = np.array(normals)

In [ ]:
elastic_rods_dir = '../../../../elastic_rods/python/'
weaving_dir = '../../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, optimization_visualization_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
importlib.reload(optimization_visualization_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)

from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, contact_optimization, get_double_side_view, show_selected_joints, highlight_rod_and_joint, get_fixed_boundary_joint, get_gravity_forces)
import vis.fields
import matplotlib.cm as cm
import time
from optimization_visualization_helper import (compute_visualization_data_from_raw_data, get_objective_components_stage1, get_objective_components_stage2, get_objective_components_stage3, set_figure_label_and_limit, Visualization_Setting, plot_objective, plot_ribbon_component_analysis)

In [ ]:
import parallelism
parallelism.set_max_num_tbb_threads(12)
parallelism.set_hessian_assembly_num_threads(4)
parallelism.set_gradient_assembly_num_threads(4)

In [ ]:
rod_length = 0.3534025445286393
width = rod_length / 75 * 5
thickness = width / 5 * 0.1
print(width, thickness)
scale = 1
io = InputOrganizer('dna_thick_{}'.format(scale), thickness, width, weaving_dir)
io.SUBDIVISION_RESOLUTION = 50

In [ ]:
5 / rod_length

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-7
OPTS.verbose = 1
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False
rw = 5
sw = 20
drw = 0.0001
dsw = 0.01

In [ ]:
# import pickle, gzip
# pickle.dump(curved_linkage, gzip.open('{}_stage_2_20210420.pkl.gz'.format(io.MODEL_NAME), 'w'))

In [ ]:
with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION, input_joint_normals = normals)
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view = linkage_vis.LinkageViewer(curved_linkage)
curved_linkage_view.show()

In [ ]:
bottomColor =[79/255., 158/255., 246/255.]
topColor =[0.5, 0.5, 0.5]
heights = curved_linkage.visualizationGeometryHeightColors()
colors = np.take(np.array([bottomColor, topColor]), heights < heights.mean(), axis=0)

In [ ]:
# renderCam = curved_linkage_view.getCameraParams()

# orender = curved_linkage_view.offscreenRenderer(width=2048, height=2048)
# orender.setCameraParams(renderCam)
# orender.render()
# orender.save('dna_top.png')

In [ ]:
curved_linkage.attraction_weight = 1e-5

In [ ]:
curved_linkage.scaleJointWeights(1, 1, [])

In [ ]:
iterateData = stage_1_optimization(curved_linkage, drw, dsw, curved_linkage_view)

In [ ]:
# E0 = curved_linkage.designParameterSolve_energy()
# l0 = curved_linkage.totalRestLength()

In [ ]:
# iterateData, dpo = stage_1_optimization(curved_linkage, drw, dsw, curved_linkage_view, E0 = E0, l0 = l0, niter = 20)

In [ ]:
# import cross_section_scaling
# importlib.reload(cross_section_scaling)
# cross_section = io.RIBBON_CS
# width_scale = [0.5, 2]
# cross_section_scaling.apply_density_based_cross_sections(curved_linkage, elastic_rods.CrossSection.construct('rectangle', 2000, 0.3, [cross_section[0], width_scale[0] * cross_section[1]]), elastic_rods.CrossSection.construct('rectangle', 2000, 0.3, [cross_section[0], width_scale[1] * cross_section[1]]))


In [ ]:
curved_linkage_view.update(scalarField = colors)

In [ ]:
fixed_boundary_joints = get_fixed_boundary_joint(curved_linkage)

In [ ]:

# z
# fixed_boundary_joints = np.array([[curved_linkage.dofOffsetForJoint(i) + j for j in range(3)] for i in [1, 6, 3]]).flatten()
# y
# fixed_boundary_joints = np.array([[curved_linkage.dofOffsetForJoint(i) + j for j in range(3)] for i in [3, 8, 1]]).flatten()

In [ ]:
fixed_boundary_joints

In [ ]:
def eqm_callback(prob, i):
    curved_linkage_view.update(scalarField = colors)

In [ ]:
driver=curved_linkage.centralJoint()
jdo = curved_linkage.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
elastic_rods.compute_equilibrium(curved_linkage, fixedVars=fixedVars, callback = eqm_callback, options = OPTS)

In [ ]:
ribbons = order_segments_by_ribbons(curved_linkage)


In [ ]:
def get_opposite_joint_type(curr_type):
    if curr_type == curved_linkage.joint(0).Type.A_OVER_B:
        return curved_linkage.joint(0).Type.B_OVER_A
    return curved_linkage.joint(0).Type.A_OVER_B
import structural_analysis

In [ ]:
structure_view = get_structure_analysis_view(curved_linkage)
structure_view.show()

In [ ]:
cfm = structural_analysis.weavingCrossingForceMagnitudes(curved_linkage, True)
if np.sum(cfm[:, 0]>0) > 0:
    print("Flipping crossing: ", np.argsort(cfm[:, 0] > 0)[-1 * np.sum(cfm[:, 0] > 0):])
    for i in np.argsort(cfm[:, 0] > 0)[-1 * np.sum(cfm[:, 0] > 0):]:
        curved_linkage.joint(i).type = get_opposite_joint_type(curved_linkage.joint(i).type)
    elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS)
    curved_linkage_view.update()

In [ ]:
# curved_save_tgt_joint_pos = curved_linkage.jointPositions()

In [ ]:
optimizer = initialize_stage_2_optimizer(curved_linkage, io.SURFACE_PATH, curved_save_tgt_joint_pos, curved_linkage_view, rw, sw, inner_gradTol = 1e-6)

In [ ]:
optimizer.scaleJointWeights(1, 1, [])

In [ ]:
optimizer.set_holdClosestPointsFixed(True)

In [ ]:
optimizer, opt_iterateData, _ = stage_2_optimization(optimizer, curved_linkage, io.SURFACE_PATH, curved_save_tgt_joint_pos, curved_linkage_view, -1, [], no_surface = False)

In [ ]:
save_path = os.getcwd()
write_all_output(curved_linkage, io.SUBDIVISION_RESOLUTION, io.MODEL_NAME, io.RIBBON_NAME, False, 100, min(io.RIBBON_CS), target_width = 5, flip_angles = False)
os.chdir(save_path)

### Gravity

In [ ]:
gravity_linkage = curved_linkage
gravity_linkage_view = linkage_vis.LinkageViewer(gravity_linkage)
gravity_linkage_view.show()

In [ ]:
np.argsort(gravity_linkage.jointPositions().reshape((curved_linkage.numJoints(), 3))[:, 2])

In [ ]:
fixed_boundary_joints = get_fixed_boundary_joint(gravity_linkage)
fixed_boundary_joints = np.array([[gravity_linkage.dofOffsetForJoint(i) + j for j in range(3)] for i in [20, 21, 22]]).flatten()

In [ ]:
OPTS.niter = 1000000

In [ ]:
external_forces = get_gravity_forces(gravity_linkage, io)

In [ ]:
def eqm_callback(prob, i):
    gravity_linkage_view.update()

In [30]:
elastic_rods.compute_equilibrium(gravity_linkage, external_forces, fixedVars=fixed_boundary_joints, callback = eqm_callback, options = OPTS)

1272	1.96633e-07	1.94043e-06	1.94043e-06	1	0
1273	1.96609e-07	3.59346e-06	3.59346e-06	1	0
1274	1.96594e-07	2.6904e-06	2.6904e-06	0.5	0
1275	1.9657e-07	4.07343e-06	4.07343e-06	1	0
1276	1.9654e-07	8.23348e-07	8.23348e-07	0.5	0
1277	1.96539e-07	5.77404e-06	5.77404e-06	1	0
1278	1.96503e-07	1.06864e-06	1.06864e-06	0.25	0
1279	1.96501e-07	4.63681e-06	4.63681e-06	1	0
1280	1.96478e-07	1.77572e-06	1.77572e-06	0.25	0
1281	1.96472e-07	3.54178e-06	3.54178e-06	1	0
1282	1.96451e-07	1.02093e-06	1.02093e-06	0.25	0
1283	1.96444e-07	2.02911e-06	2.02911e-06	1	0
1284	1.96436e-07	2.46203e-06	2.46203e-06	1	0
Computing negative curvature direction for scaled tau = 1.81996
1285	1.96427e-07	6.68093e-06	6.68093e-06	0.0625	1
1286	1.96425e-07	6.32049e-06	6.32049e-06	0.25	0
1287	1.96409e-07	4.92402e-06	4.92402e-06	1	0
1288	1.96385e-07	1.10468e-06	1.10468e-06	0.25	0
1289	1.9638e-07	1.68986e-06	1.68986e-06	1	0
1290	1.96373e-07	2.25934e-06	2.25934e-06	0.5	0
1291	1.9636e-07	2.24481e-06	2.24481e-06	1	0
1292	1.96346e-07